## 划分训练集和测试集

### 1.随机抽样

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split( data, test_size=0.2, random_state=42)

test_size:测试集大小;  random_state:随机种子

### 2.分层抽样

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(data, data["income_cat"]):
    strat_train_set = data.loc[train_index]
    strat_test_set = data.loc[test_index]

n_splits是将训练数据分成train/test对的组数，可根据需要进行设置，默认为10(此处n_splits=1)